# Data Challenge - La ricerca "Stati d'Animo"

## IMPORT LIBRARY

In [1]:
import pandas as pd
import numpy as np

from datetime import datetime, timedelta

## IMPORT DATASET

In [2]:
# load df with values
indagine = pd.read_excel('data/dataset_indagine_stati_animo.xlsx', sheet_name='Values')
profilo = pd.read_excel('data/dataset_profilo_intervistati.xlsx', sheet_name='Values')

# load df with labels
indagine_labels = pd.read_excel('data/dataset_indagine_stati_animo.xlsx', sheet_name='Labels')
profilo_labels = pd.read_excel('data/dataset_profilo_intervistati.xlsx', sheet_name='Labels')

# load df with structure
indagine_lookup = pd.read_excel('data/dataset_indagine_stati_animo.xlsx', sheet_name='Struttura dei dati', skiprows=5, nrows=indagine.shape[1])
profilo_lookup = pd.read_excel('data/dataset_profilo_intervistati.xlsx', sheet_name='Struttura dei dati', skiprows=5, nrows=profilo.shape[1])


## DATASET INDAGINE E PROFILO

In [3]:
df_indagine= indagine.copy()
df_profilo = profilo.copy()

df_indagine_labels = indagine_labels.copy()
df_profilo_labels = profilo_labels.copy()

df_indagine_lookup = indagine_lookup.copy()
df_profilo_lookup = profilo_lookup.copy()

### DATA CLEANING

#### DF INDAGINE

In [4]:
# cleaning of the Etichetta variable in the Indagine data structure DataFrame --> to generate the Domanda variable
df_indagine_lookup['Domanda'] = df_indagine_lookup['Etichetta'].apply(lambda st: st[st.find("Question: ")+10:np.min([st.find("]"), st.find("<span ")])] if st.find("Question: ") >= 0 else st)
df_indagine_lookup['Domanda'] = df_indagine_lookup['Domanda'].apply(lambda st: st[:st.find("<span ")] if st.find("<span ") >= 0 else st)
df_indagine_lookup['Domanda'] = df_indagine_lookup['Domanda'].apply(lambda st: st[:st.find("<br />")] if st.find("<br />") >= 0 else st)

# cleaning of the Etichetta variable in the Indagine data structure DataFrame --> to generate the Topic variable
df_indagine_lookup['Topic'] = df_indagine_lookup['Etichetta'].apply(lambda st: st[st.find("Topic: ")+7:st.find("Question: ")-1] if st.find("Topic: ") >= 0 else st[:st.find("[Question: ")-1] if st.find("[Question: ") >= 0 else "")

# mapping between question code and description
df_indagine_lookup = df_indagine_lookup[1:-28]
df_indagine_lookup = df_indagine_lookup[['Variabile', 'Domanda', 'Topic']]
df_indagine_lookup = df_indagine_lookup.reset_index(drop=True)

# transposing the data from horizontal to vertical
df_indagine_transposed = pd.melt(df_indagine[df_indagine.columns[0:-28]], id_vars='ID_Rispondente', var_name='domanda', value_name='values_risposta')

# adding columns with mapping between question code and description
df_indagine_transposed = df_indagine_transposed.merge(df_indagine_lookup, how='left', left_on='domanda', right_on='Variabile')
df_indagine_transposed = df_indagine_transposed.drop(columns=['Variabile'])

In [5]:
df_indagine_transposed.head()

,ID_Rispondente,domanda,values_risposta,Domanda,Topic
0,3936511661,B11_Q62_A_1,5,Di seguito trovi una serie di parole che indic...,Attivo/ intraprendente
1,164860483,B11_Q62_A_1,3,Di seguito trovi una serie di parole che indic...,Attivo/ intraprendente
2,221250719,B11_Q62_A_1,5,Di seguito trovi una serie di parole che indic...,Attivo/ intraprendente
3,89508748,B11_Q62_A_1,3,Di seguito trovi una serie di parole che indic...,Attivo/ intraprendente
4,98143773,B11_Q62_A_1,2,Di seguito trovi una serie di parole che indic...,Attivo/ intraprendente


In [6]:
# spliting positive and negative "topics" in 2 lists
item_positivi = ['Attivo/ intraprendente', 'Ispirato/ Stimolato', 'Sveglio/ Lucido', 'Determinato/ Risoluto', 'Attento/ Sul pezzo', 'Gioia', 'Fiducia', 'Speranza', 'Protezione/ Sicurezza', 'Armonia', 'Serenità']
item_negativi = ['Turbato/ Agitato','Ostile/ Maldisposto', 'Imbarazzato/ Umiliato',  'Nervoso', 'Spaventato/ Impaurito', 'Tristezza', 'Indifferenza', 'Incomprensione', 'Noia','Rabbia', 'Ansia/ Inquietudine']

# adding a new column to categorize positive and negative "topics"
df_indagine_transposed['tipologia_topic'] = df_indagine_transposed['Topic'].apply(lambda x: 'positivo' if x in item_positivi else ('negativo' if x in item_negativi else ''))

# adding a new column with the value answers and +/- sign according the positive/negative topics category
df_indagine_transposed['values_risposta_segno'] = df_indagine_transposed.apply(lambda row: row['values_risposta'] * -1 if row['tipologia_topic'] == 'negativo' else row['values_risposta'], axis=1)

In [7]:
df_indagine_transposed.head()

,ID_Rispondente,domanda,values_risposta,Domanda,Topic,tipologia_topic,values_risposta_segno
0,3936511661,B11_Q62_A_1,5,Di seguito trovi una serie di parole che indic...,Attivo/ intraprendente,positivo,5
1,164860483,B11_Q62_A_1,3,Di seguito trovi una serie di parole che indic...,Attivo/ intraprendente,positivo,3
2,221250719,B11_Q62_A_1,5,Di seguito trovi una serie di parole che indic...,Attivo/ intraprendente,positivo,5
3,89508748,B11_Q62_A_1,3,Di seguito trovi una serie di parole che indic...,Attivo/ intraprendente,positivo,3
4,98143773,B11_Q62_A_1,2,Di seguito trovi una serie di parole che indic...,Attivo/ intraprendente,positivo,2


In [8]:
# making a df with questions code and question names
df_domande = df_indagine_transposed[['domanda','Domanda']].drop_duplicates().reset_index(drop=True)
df_domande.domanda = df_domande.domanda.apply(lambda x: f'{x.split("_")[1:4][0]}')
df_domande = df_domande.drop_duplicates()
df_domande = df_domande.reset_index(drop=True)
df_domande = df_domande.rename(columns={'domanda':'cd_domande'})

In [9]:
df_domande.head()

,cd_domande,Domanda
0,Q62,Di seguito trovi una serie di parole che indic...
1,Q63,"Quando pensi al futuro, fra un anno ti vedi pi..."
2,Q64,Come vedi la tua vita nei prossimi 12 mesi?Qua...
3,Q65,Quali sono i tre stati d'animo principali che ...
4,Q66,Quali sono i tre stati d'animo principali che ...


In [10]:
# adding the column 'cd_domande'
df_indagine_transposed = df_indagine_transposed.merge(df_domande, how='left', left_on='Domanda', right_on='Domanda')

In [11]:
df_indagine_transposed.head()

,ID_Rispondente,domanda,values_risposta,Domanda,Topic,tipologia_topic,values_risposta_segno,cd_domande
0,3936511661,B11_Q62_A_1,5,Di seguito trovi una serie di parole che indic...,Attivo/ intraprendente,positivo,5,Q62
1,164860483,B11_Q62_A_1,3,Di seguito trovi una serie di parole che indic...,Attivo/ intraprendente,positivo,3,Q62
2,221250719,B11_Q62_A_1,5,Di seguito trovi una serie di parole che indic...,Attivo/ intraprendente,positivo,5,Q62
3,89508748,B11_Q62_A_1,3,Di seguito trovi una serie di parole che indic...,Attivo/ intraprendente,positivo,3,Q62
4,98143773,B11_Q62_A_1,2,Di seguito trovi una serie di parole che indic...,Attivo/ intraprendente,positivo,2,Q62


In [12]:
# checking the unique in cd_domande
sorted(df_indagine_transposed.cd_domande.unique())

['Q62', 'Q63', 'Q64', 'Q65', 'Q66', 'Q67', 'Q69', 'Q70', 'Q71']

In [13]:
# adding a column to categorize questions
df_indagine_transposed['tipologia_domanda'] = df_indagine_transposed['cd_domande'].apply(lambda x: 'vita personale' if x in ['Q62', 'Q63'] else ('prossimi 12 mesi' if x in ['Q64'] else ('contesto' if x in ['Q65', 'Q66','Q67', 'Q69'] else 'generale')))

In [14]:
# checking the unique in tipologia_domanda
df_indagine_transposed['tipologia_domanda'].unique()

array(['vita personale', 'prossimi 12 mesi', 'contesto', 'generale'],
      dtype=object)

In [15]:
df_indagine_transposed.head()

,ID_Rispondente,domanda,values_risposta,Domanda,Topic,tipologia_topic,values_risposta_segno,cd_domande,tipologia_domanda
0,3936511661,B11_Q62_A_1,5,Di seguito trovi una serie di parole che indic...,Attivo/ intraprendente,positivo,5,Q62,vita personale
1,164860483,B11_Q62_A_1,3,Di seguito trovi una serie di parole che indic...,Attivo/ intraprendente,positivo,3,Q62,vita personale
2,221250719,B11_Q62_A_1,5,Di seguito trovi una serie di parole che indic...,Attivo/ intraprendente,positivo,5,Q62,vita personale
3,89508748,B11_Q62_A_1,3,Di seguito trovi una serie di parole che indic...,Attivo/ intraprendente,positivo,3,Q62,vita personale
4,98143773,B11_Q62_A_1,2,Di seguito trovi una serie di parole che indic...,Attivo/ intraprendente,positivo,2,Q62,vita personale


In [16]:
# dropping question categories that's don't needed
df = df_indagine_transposed[df_indagine_transposed['tipologia_domanda']=='vita personale']

In [17]:
# checking the unique in tipologia_domanda
df_indagine_transposed['tipologia_domanda'].unique()

array(['vita personale', 'prossimi 12 mesi', 'contesto', 'generale'],
      dtype=object)

In [18]:
# checking the unique in tipologia_domanda
df['cd_domande'].unique()

array(['Q62', 'Q63'], dtype=object)

In [19]:
# making a df with a column with a score about moods for each interviewed-question
df = df.groupby(['ID_Rispondente','tipologia_domanda','cd_domande'])['values_risposta_segno'].sum().reset_index()
df.head()

,ID_Rispondente,tipologia_domanda,cd_domande,values_risposta_segno
0,256848,vita personale,Q62,11
1,256848,vita personale,Q63,7
2,337988,vita personale,Q62,9
3,337988,vita personale,Q63,1
4,371674,vita personale,Q62,2


In [20]:
# adding the column "peso"
df = df.merge(df_indagine_labels[['ID_Rispondente','peso']], how='left', left_on='ID_Rispondente', right_on='ID_Rispondente')

# adding a new column with the value answers weighted
df['values_risposta_pesata'] = df['values_risposta_segno'] * df['peso']

In [21]:
df.head()

,ID_Rispondente,tipologia_domanda,cd_domande,values_risposta_segno,peso,values_risposta_pesata
0,256848,vita personale,Q62,11,0.218331,2.401636
1,256848,vita personale,Q63,7,0.218331,1.528314
2,337988,vita personale,Q62,9,1.211879,10.906914
3,337988,vita personale,Q63,1,1.211879,1.211879
4,371674,vita personale,Q62,2,0.218331,0.436661


In [22]:
# renaming columns
df = df.rename(columns={'tipologia_domanda':'domanda_tipologia',
                        'cd_domande': 'domanda',
                        'values_risposta_segno':'punteggio',
                        'values_risposta_pesata':'punteggio_ponderato'})

In [23]:
df.head()

,ID_Rispondente,domanda_tipologia,domanda,punteggio,peso,punteggio_ponderato
0,256848,vita personale,Q62,11,0.218331,2.401636
1,256848,vita personale,Q63,7,0.218331,1.528314
2,337988,vita personale,Q62,9,1.211879,10.906914
3,337988,vita personale,Q63,1,1.211879,1.211879
4,371674,vita personale,Q62,2,0.218331,0.436661


In [24]:
# making a dictionary to use as mapping
mappa_valori = {'Q65':'casa', 'Q66':'vicinato', 'Q67':'Italia', 'Q69':'mondo', 'Q62':'oggi', 'Q63':'futuro'}

# replacing question code with question short name
df['domanda'] = df['domanda'].replace(mappa_valori)

In [25]:
print(df.shape)
df.head(6)

(16256, 6)


,ID_Rispondente,domanda_tipologia,domanda,punteggio,peso,punteggio_ponderato
0,256848,vita personale,oggi,11,0.218331,2.401636
1,256848,vita personale,futuro,7,0.218331,1.528314
2,337988,vita personale,oggi,9,1.211879,10.906914
3,337988,vita personale,futuro,1,1.211879,1.211879
4,371674,vita personale,oggi,2,0.218331,0.436661
5,371674,vita personale,futuro,1,0.218331,0.218331


In [26]:
# checking descriptive statistics for questions about 'vita personale'
df[df.domanda_tipologia=='vita personale'][['punteggio', 'punteggio_ponderato']].describe()

,punteggio,punteggio_ponderato
count,16256.000000,16256.000000
mean,5.367987,5.338300
std,6.630201,10.440425
min,-20.000000,-98.248764
25%,0.000000,0.000000
50%,5.000000,2.524249
75%,10.000000,7.536993
max,20.000000,109.165293


In [27]:
# transposing the data from vertical to horizontal
df_pivot_1 = df.pivot_table(index='ID_Rispondente', columns='domanda', values='punteggio', aggfunc='first').reset_index()
df_pivot_1 = df_pivot_1.rename_axis('', axis='columns')

In [28]:
print(df_pivot_1.shape)
df_pivot_1.head()

(8128, 3)


,ID_Rispondente,futuro,oggi
0,256848,7,11
1,337988,1,9
2,371674,1,2
3,379221,13,10
4,514191,10,9


In [29]:
# making a duplicate column of 'domanda' but adding the string 'pesato' at the end
df['domanda_pesato'] = df.domanda.apply(lambda x: x + '_pesato' )

In [30]:
# transposing the data from vertical to horizontal
df_pivot_2 = df.pivot_table(index='ID_Rispondente', columns='domanda_pesato', values='punteggio_ponderato', aggfunc='first').reset_index()
df_pivot_2 = df_pivot_2.rename_axis('', axis='columns')

In [31]:
print(df_pivot_2.shape)
df_pivot_2.head()

(8128, 3)


,ID_Rispondente,futuro_pesato,oggi_pesato
0,256848,1.528314,2.401636
1,337988,1.211879,10.906914
2,371674,0.218331,0.436661
3,379221,38.273890,29.441454
4,514191,20.472955,18.425660


In [32]:
# left join the 2 pivot tables
df = df_pivot_1.merge(df_pivot_2, how='left', left_on='ID_Rispondente', right_on='ID_Rispondente')
df.head()

,ID_Rispondente,futuro,oggi,futuro_pesato,oggi_pesato
0,256848,7,11,1.528314,2.401636
1,337988,1,9,1.211879,10.906914
2,371674,1,2,0.218331,0.436661
3,379221,13,10,38.273890,29.441454
4,514191,10,9,20.472955,18.425660


In [33]:
# filtering the df_labels with only the columns I need to add
df_indagine_labels = df_indagine_labels.iloc[:, [0] + list(range(82, 97))]

# adding the df_labels's columns to the df
df = df.merge(df_indagine_labels, how='left', left_on='ID_Rispondente', right_on='ID_Rispondente')

In [34]:
df.head()

,ID_Rispondente,futuro,oggi,futuro_pesato,oggi_pesato,B11_tempo,B11_passo,B11_mese,Sesso,Age,Età_Fasce,p_Q9,CAP,Comune,Provincia,Regione,Abitanti_Classi,Numero_di_abitanti,Area_Nielsen,Area_Nielsen_OLD
0,256848,7,11,1.528314,2.401636,98.013,8.167750,44976.441655,Una donna,39.0,35-44,Laurea o dottorato,8100.0,NUORO,NU,SAR,Tra 30 e 100mila,36678.0,Area 3,Isole
1,337988,1,9,1.211879,10.906914,145.659,12.138250,45058.745671,Un uomo,56.0,55-64,Diploma di scuola media inferiore,24057.0,MARTINENGO,BG,LOM,Tra 10 e 30mila,10524.0,Area 1,Nordovest
2,371674,1,2,0.218331,0.436661,143.891,11.990917,44985.510521,Una donna,46.0,45-54,Laurea o dottorato,34139.0,TRIESTE,TS,FVG,Più di 100mila,211184.0,Area 2,Nordest
3,379221,13,10,38.273890,29.441454,237.097,19.758083,45038.741539,Un uomo,77.0,65+,Diploma di scuola media superiore,85044.0,LAURIA,PZ,BAS,Tra 10 e 30mila,13801.0,Area 4,Sud
4,514191,10,9,20.472955,18.425660,258.143,21.511917,44995.776377,Un uomo,73.0,65+,Diploma di scuola media inferiore,70123.0,BARI,BA,PUG,Più di 100mila,316532.0,Area 4,Sud


In [35]:
# adding column 'peso'
df_peso = indagine.copy()
df_peso = df_peso.iloc[:,[0]+[107]]
df = df.merge(df_peso, how='left', left_on='ID_Rispondente', right_on='ID_Rispondente')

In [36]:
df_peso.head()

,ID_Rispondente,peso
0,3936511661,0.581195
1,164860483,5.458265
2,221250719,5.458265
3,89508748,5.458265
4,98143773,5.458265


In [37]:
df.head()

,ID_Rispondente,futuro,oggi,futuro_pesato,oggi_pesato,B11_tempo,B11_passo,B11_mese,Sesso,Age,...,p_Q9,CAP,Comune,Provincia,Regione,Abitanti_Classi,Numero_di_abitanti,Area_Nielsen,Area_Nielsen_OLD,peso
0,256848,7,11,1.528314,2.401636,98.013,8.167750,44976.441655,Una donna,39.0,...,Laurea o dottorato,8100.0,NUORO,NU,SAR,Tra 30 e 100mila,36678.0,Area 3,Isole,0.218331
1,337988,1,9,1.211879,10.906914,145.659,12.138250,45058.745671,Un uomo,56.0,...,Diploma di scuola media inferiore,24057.0,MARTINENGO,BG,LOM,Tra 10 e 30mila,10524.0,Area 1,Nordovest,1.211879
2,371674,1,2,0.218331,0.436661,143.891,11.990917,44985.510521,Una donna,46.0,...,Laurea o dottorato,34139.0,TRIESTE,TS,FVG,Più di 100mila,211184.0,Area 2,Nordest,0.218331
3,379221,13,10,38.273890,29.441454,237.097,19.758083,45038.741539,Un uomo,77.0,...,Diploma di scuola media superiore,85044.0,LAURIA,PZ,BAS,Tra 10 e 30mila,13801.0,Area 4,Sud,2.944145
4,514191,10,9,20.472955,18.425660,258.143,21.511917,44995.776377,Un uomo,73.0,...,Diploma di scuola media inferiore,70123.0,BARI,BA,PUG,Più di 100mila,316532.0,Area 4,Sud,2.047296


In [38]:
# converting the survey date into date format
df['B11_mese'] = df['B11_mese'].apply(lambda x: (datetime(1899, 12, 30) + timedelta(days=x)).strftime('%d/%m/%Y'))
df['B11_mese'] = pd.to_datetime(df['B11_mese'], dayfirst=True)

In [39]:
# renaming columns
df = df.rename(columns={'B11_tempo':'Tempo compilazione',
                        'B11_passo':'Secondi per risposta',
                        'B11_mese':'Data compilazione',
                        'p_Q9':'Istruzione'})

In [40]:
# checking the columns Q70 answers
df_indagine_transposed[df_indagine_transposed['cd_domande']=='Q70'][['ID_Rispondente','values_risposta']].head()

,ID_Rispondente,values_risposta
625856,3936511661,2
625857,164860483,3
625858,221250719,3
625859,89508748,3
625860,98143773,3


In [41]:
# adding a columns with Q70 answers
df = df.merge(df_indagine_transposed[df_indagine_transposed['cd_domande']=='Q70'][['ID_Rispondente','values_risposta']], how='left', left_on='ID_Rispondente', right_on='ID_Rispondente')
df = df.rename(columns={'values_risposta':'soddisfazione_vita'})

# adding a columns with Q70 weighted answers 
# df['soddisfazione_vita_pesato'] = df['soddisfazione_vita'] * df['peso'] #added in tableau with a calculated field

In [42]:
print(df.shape)
df.head()

(8128, 22)


,ID_Rispondente,futuro,oggi,futuro_pesato,oggi_pesato,Tempo compilazione,Secondi per risposta,Data compilazione,Sesso,Age,...,CAP,Comune,Provincia,Regione,Abitanti_Classi,Numero_di_abitanti,Area_Nielsen,Area_Nielsen_OLD,peso,soddisfazione_vita
0,256848,7,11,1.528314,2.401636,98.013,8.167750,2023-02-19,Una donna,39.0,...,8100.0,NUORO,NU,SAR,Tra 30 e 100mila,36678.0,Area 3,Isole,0.218331,3
1,337988,1,9,1.211879,10.906914,145.659,12.138250,2023-05-12,Un uomo,56.0,...,24057.0,MARTINENGO,BG,LOM,Tra 10 e 30mila,10524.0,Area 1,Nordovest,1.211879,3
2,371674,1,2,0.218331,0.436661,143.891,11.990917,2023-02-28,Una donna,46.0,...,34139.0,TRIESTE,TS,FVG,Più di 100mila,211184.0,Area 2,Nordest,0.218331,2
3,379221,13,10,38.273890,29.441454,237.097,19.758083,2023-04-22,Un uomo,77.0,...,85044.0,LAURIA,PZ,BAS,Tra 10 e 30mila,13801.0,Area 4,Sud,2.944145,3
4,514191,10,9,20.472955,18.425660,258.143,21.511917,2023-03-10,Un uomo,73.0,...,70123.0,BARI,BA,PUG,Più di 100mila,316532.0,Area 4,Sud,2.047296,4


#### DF PROFILO

In [43]:
# cleaning of the Etichetta variable in the Profilo data structure DataFrame --> to generate the Domanda variable
df_profilo_lookup['Domanda'] = df_profilo_lookup['Etichetta'].apply(lambda st: st[st.find("Question: ")+10:np.min([st.find("]"), st.find("<br />")])] if st.find("Question: ") >= 0 else st)

# cleaning of the Etichetta variable in the Profilo data structure DataFrame --> to generate the Topic variable
df_profilo_lookup['Topic'] = df_profilo_lookup['Etichetta'].apply(lambda st: st[st.find("/> ")+3:st.find(" [Question:")] if st.find("<img src=") >= 0 else st[0:st.find(" [Question:")] if st.find(" [Question:]") else "")
df_profilo_lookup['Topic'] = df_profilo_lookup['Topic'].apply(lambda st: st[st.find("[Topic: ")+8:st.find(" Question:")] if st.find("[Topic: ") >= 0 else st)
df_profilo_lookup['Topic'] = df_profilo_lookup['Topic'].apply(lambda st: st[0:st.find("<br />")] if st.find("<br />") >= 0 else st)

# melting the profilo_value DF --> transposing the data from horizontal to vertical
df_profilo_transposed = pd.melt(df_profilo, id_vars='ID_Rispondente', var_name='domanda', value_name='values_risposta')

# melting the profilo_label DF --> transposing the data from horizontal to vertical
df_profilo_labels_transposed = pd.melt(df_profilo_labels, id_vars='ID_Rispondente', var_name='domanda', value_name='labels_risposta')

# inner joining the df_profilo_labels_transposed and df_profilo_transposed 
df_profilo_transposed = pd.merge(df_profilo_labels_transposed, df_profilo_transposed, how='inner', on=['ID_Rispondente','domanda'])

# left joining the df_profilo_transposed with the df_profilo_lookup (data mapping)
df_profilo_transposed = pd.merge(df_profilo_transposed, df_profilo_lookup[['Variabile', 'Domanda', 'Topic']], how='left', left_on='domanda', right_on='Variabile')

# renaming the variables and dropping unwanted columns
df_profilo_transposed.columns = ['idRispondente', 'cdDomanda', 'cdLabelsRisposta', 'cdValuesRisposta', 'Variabile', 'nmDomanda', 'nmRisposta']
df_profilo_transposed = df_profilo_transposed[['idRispondente', 'cdDomanda', 'cdLabelsRisposta', 'cdValuesRisposta', 'nmDomanda', 'nmRisposta']]

# dropping rows with no answers
df_profilo_transposed = df_profilo_transposed.dropna(subset=['cdValuesRisposta'])

In [44]:
print(df_profilo_transposed.shape)
df_profilo_transposed.head(5)

(2239416, 6)


,idRispondente,cdDomanda,cdLabelsRisposta,cdValuesRisposta,nmDomanda,nmRisposta
0,256848,B01_Q64_1,Selected,1.0,Quali di questi strumenti di social networking...,Twitter
1,337988,B01_Q64_1,Selected,1.0,Quali di questi strumenti di social networking...,Twitter
2,369486,B01_Q64_1,Selected,1.0,Quali di questi strumenti di social networking...,Twitter
3,371674,B01_Q64_1,Selected,1.0,Quali di questi strumenti di social networking...,Twitter
4,379221,B01_Q64_1,Selected,1.0,Quali di questi strumenti di social networking...,Twitter


In [45]:
# adding new column with their first 6 letters of cdDomanda
df_profilo_transposed['cdDomanda_short'] = df_profilo_transposed.cdDomanda.apply(lambda x: f'{x[0:7]}')

In [46]:
df_profilo_transposed.head()

,idRispondente,cdDomanda,cdLabelsRisposta,cdValuesRisposta,nmDomanda,nmRisposta,cdDomanda_short
0,256848,B01_Q64_1,Selected,1.0,Quali di questi strumenti di social networking...,Twitter,B01_Q64
1,337988,B01_Q64_1,Selected,1.0,Quali di questi strumenti di social networking...,Twitter,B01_Q64
2,369486,B01_Q64_1,Selected,1.0,Quali di questi strumenti di social networking...,Twitter,B01_Q64
3,371674,B01_Q64_1,Selected,1.0,Quali di questi strumenti di social networking...,Twitter,B01_Q64
4,379221,B01_Q64_1,Selected,1.0,Quali di questi strumenti di social networking...,Twitter,B01_Q64


In [47]:
# renaming a column
df_profilo_transposed = df_profilo_transposed.rename(columns={'idRispondente':'ID_Rispondente'})

In [48]:
# filtring the df by the questions to analyze
df_profilo_transposed = df_profilo_transposed[df_profilo_transposed['cdDomanda'].isin(['B02_Q62','B02_Q64','B02_Q65'])]

In [49]:
# making a dictionary to use as mapping
mappa_valori_profilo = {'B02_Q62':'stato_occupazione', 'B02_Q64':'ruolo_lavoro', 'B02_Q65':'dimensione_azienda'}

# adding a column with question short name
df_profilo_transposed['categoria_domanda'] = df_profilo_transposed['cdDomanda_short'].replace(mappa_valori_profilo)

In [50]:
df_profilo_transposed.head(3)

,ID_Rispondente,cdDomanda,cdLabelsRisposta,cdValuesRisposta,nmDomanda,nmRisposta,cdDomanda_short,categoria_domanda
681209,256848,B02_Q62,Lavori a tempo pieno,1.0,In questo momento...,In questo momento..,B02_Q62,stato_occupazione
681210,337988,B02_Q62,Lavori a tempo pieno,1.0,In questo momento...,In questo momento..,B02_Q62,stato_occupazione
681211,369486,B02_Q62,Sei in pensione,5.0,In questo momento...,In questo momento..,B02_Q62,stato_occupazione


In [51]:
# renaming the variables and dropping unwanted columns
df_profilo_transposed = df_profilo_transposed.rename(columns = {'cdLabelsRisposta':'profilo_risposta_etichetta', 
                                                                'cdValuesRisposta':'profilo_risposta_valore',
                                                                'categoria_domanda':'profilo_categoria_domanda',
                                                                })
df_profilo_transposed = df_profilo_transposed[['ID_Rispondente', 'profilo_categoria_domanda', 'profilo_risposta_valore', 'profilo_risposta_etichetta']]

In [52]:
df_profilo_transposed.head()

,ID_Rispondente,profilo_categoria_domanda,profilo_risposta_valore,profilo_risposta_etichetta
681209,256848,stato_occupazione,1.0,Lavori a tempo pieno
681210,337988,stato_occupazione,1.0,Lavori a tempo pieno
681211,369486,stato_occupazione,5.0,Sei in pensione
681212,371674,stato_occupazione,1.0,Lavori a tempo pieno
681214,500385,stato_occupazione,1.0,Lavori a tempo pieno


In [53]:
# transposing the data from vertical to horizontal
df_profilo_transposed = df_profilo_transposed.pivot_table(index='ID_Rispondente', columns='profilo_categoria_domanda', values=['profilo_risposta_valore','profilo_risposta_etichetta'], aggfunc='first').reset_index()
df_profilo_transposed.columns = df_profilo_transposed.columns.map('-'.join)

In [54]:
df_profilo_transposed.head()

,ID_Rispondente-,profilo_risposta_etichetta-dimensione_azienda,profilo_risposta_etichetta-ruolo_lavoro,profilo_risposta_etichetta-stato_occupazione,profilo_risposta_valore-dimensione_azienda,profilo_risposta_valore-ruolo_lavoro,profilo_risposta_valore-stato_occupazione
0,256848,11 - 50 addetti,Quadro,Lavori a tempo pieno,4.0,3.0,1.0
1,337988,11 - 50 addetti,Altro ruolo e cioè:,Lavori a tempo pieno,4.0,6.0,1.0
2,369486,NaN,NaN,Sei in pensione,NaN,NaN,5.0
3,371674,11 - 50 addetti,Impiegato,Lavori a tempo pieno,4.0,4.0,1.0
4,500385,11 - 50 addetti,Impiegato,Lavori a tempo pieno,4.0,4.0,1.0


In [55]:
# renaming the columns
df_profilo_transposed = df_profilo_transposed.rename(columns = {'profilo_risposta_etichetta-dimensione_azienda':'dimensione_azienda_etichetta',
                                                                'profilo_risposta_etichetta-ruolo_lavoro':'ruolo_lavoro_etichetta',
                                                                'profilo_risposta_etichetta-stato_occupazione':'stato_occupazione_etichetta',
                                                                'profilo_risposta_valore-dimensione_azienda':'dimensione_azienda_valore',
                                                                'profilo_risposta_valore-ruolo_lavoro':'ruolo_lavoro_valore',
                                                                'profilo_risposta_valore-stato_occupazione':'stato_occupazione_valore',
                                                                'ID_Rispondente-':'ID_Rispondente'
                                                                })

In [56]:
print(df_profilo_transposed.shape)
df_profilo_transposed.head()

(10300, 7)


,ID_Rispondente,dimensione_azienda_etichetta,ruolo_lavoro_etichetta,stato_occupazione_etichetta,dimensione_azienda_valore,ruolo_lavoro_valore,stato_occupazione_valore
0,256848,11 - 50 addetti,Quadro,Lavori a tempo pieno,4.0,3.0,1.0
1,337988,11 - 50 addetti,Altro ruolo e cioè:,Lavori a tempo pieno,4.0,6.0,1.0
2,369486,NaN,NaN,Sei in pensione,NaN,NaN,5.0
3,371674,11 - 50 addetti,Impiegato,Lavori a tempo pieno,4.0,4.0,1.0
4,500385,11 - 50 addetti,Impiegato,Lavori a tempo pieno,4.0,4.0,1.0


### MERGE 

INDAGINE + PROFILO

In [57]:
# adding a df_profilo's columns to df_indagine
df = df.merge(df_profilo_transposed, how='left', left_on='ID_Rispondente', right_on='ID_Rispondente')

In [58]:
print(df.shape)
df.head()

(8128, 28)


,ID_Rispondente,futuro,oggi,futuro_pesato,oggi_pesato,Tempo compilazione,Secondi per risposta,Data compilazione,Sesso,Age,...,Area_Nielsen,Area_Nielsen_OLD,peso,soddisfazione_vita,dimensione_azienda_etichetta,ruolo_lavoro_etichetta,stato_occupazione_etichetta,dimensione_azienda_valore,ruolo_lavoro_valore,stato_occupazione_valore
0,256848,7,11,1.528314,2.401636,98.013,8.167750,2023-02-19,Una donna,39.0,...,Area 3,Isole,0.218331,3,11 - 50 addetti,Quadro,Lavori a tempo pieno,4.0,3.0,1.0
1,337988,1,9,1.211879,10.906914,145.659,12.138250,2023-05-12,Un uomo,56.0,...,Area 1,Nordovest,1.211879,3,11 - 50 addetti,Altro ruolo e cioè:,Lavori a tempo pieno,4.0,6.0,1.0
2,371674,1,2,0.218331,0.436661,143.891,11.990917,2023-02-28,Una donna,46.0,...,Area 2,Nordest,0.218331,2,11 - 50 addetti,Impiegato,Lavori a tempo pieno,4.0,4.0,1.0
3,379221,13,10,38.273890,29.441454,237.097,19.758083,2023-04-22,Un uomo,77.0,...,Area 4,Sud,2.944145,3,NaN,NaN,NaN,NaN,NaN,NaN
4,514191,10,9,20.472955,18.425660,258.143,21.511917,2023-03-10,Un uomo,73.0,...,Area 4,Sud,2.047296,4,NaN,NaN,Sei in pensione,NaN,NaN,5.0


## DATASET REDDITO 

Fonte 2021 ISTAT  http://dati.istat.it/Index.aspx?DataSetCode=DCCV_REDNETFAMFONTERED

### IMPORT 

In [59]:
# import df
reddito_età = pd.read_csv('data_reddito_istat/2021_reddito_fascia_età.csv')
reddito_regione = pd.read_csv('data_reddito_istat/2021_reddito_regione.csv')
reddito_sesso = pd.read_csv('data_reddito_istat/2021_reddito_sesso.csv')
reddito_titolo_studio = pd.read_csv('data_reddito_istat/2021_reddito_titolo_studio.csv')

In [60]:
# cleaning data
reddito_età = reddito_età[reddito_età['T_D8']=='REDD_MEDIO_FAM']
reddito_età = reddito_età[reddito_età['Presenza affitti imputati']=='esclusi fitti imputati']
reddito_età = reddito_età[reddito_età['Territorio']!='Italia']
reddito_età = reddito_età[['Territorio','Classe di età del principale percettore', 'Value']]
reddito_età = reddito_età.reset_index(drop=True)
reddito_età.head(3)

,Territorio,Classe di età del principale percettore,Value
0,Nord-ovest,fino a 35 anni,33796
1,Nord-est,fino a 35 anni,34486
2,Centro,fino a 35 anni,28599


In [61]:
# cleaning data
reddito_regione = reddito_regione[reddito_regione['T_D8']=='REDD_MEDIO_FAM']
reddito_regione = reddito_regione[reddito_regione['Presenza affitti imputati']=='esclusi fitti imputati']
reddito_regione = reddito_regione[~reddito_regione['Territorio'].isin(['Italia','centro area metropolitana','periferia area metropolitana', 'fino a 2.000 ab.','2.001 - 10.000 ab.', '10.001 - 50.000 ab.', '50.001 ab. e più'])]
reddito_regione = reddito_regione[['Territorio','Fonte principale di reddito familiare ', 'Value']]
reddito_regione = reddito_regione.reset_index(drop=True)
reddito_regione.head(3)

,Territorio,Fonte principale di reddito familiare,Value
0,Nord-ovest,lavoro dipendente,41438.0
1,Nord-ovest,lavoro autonomo,52988.0
2,Nord-ovest,pensioni e trasferimenti pubblici,29986.0


In [62]:
# cleaning data
reddito_sesso = reddito_sesso[reddito_sesso['T_D8']=='REDD_MEDIO_FAM']
reddito_sesso = reddito_sesso[reddito_sesso['Presenza affitti imputati']=='esclusi fitti imputati']
reddito_sesso = reddito_sesso[reddito_sesso['Territorio']!='Italia']
reddito_sesso = reddito_sesso[['Territorio','Sesso del principale percettore ', 'Value']]
reddito_sesso = reddito_sesso.reset_index(drop=True)
reddito_sesso.head(3)

,Territorio,Sesso del principale percettore,Value
0,Nord-ovest,maschi,42031
1,Nord-ovest,femmine,30933
2,Nord-est,maschi,42468


In [63]:
# cleaning data
reddito_titolo_studio = reddito_titolo_studio[reddito_titolo_studio['T_D8']=='REDD_MEDIO_FAM']
reddito_titolo_studio = reddito_titolo_studio[reddito_titolo_studio['Presenza affitti imputati']=='esclusi fitti imputati']
reddito_titolo_studio = reddito_titolo_studio[reddito_titolo_studio['Territorio']!='Italia']
reddito_titolo_studio = reddito_titolo_studio[['Territorio','Titolo di studio del principale percettore', 'Value']]
reddito_titolo_studio = reddito_titolo_studio.reset_index(drop=True)
reddito_titolo_studio.head(3)

,Territorio,Titolo di studio del principale percettore,Value
0,Nord-ovest,"licenza di scuola elementare, nessun titolo di...",21444
1,Nord-ovest,licenza di scuola media,30589
2,Nord-ovest,diploma,38923


### MERGE

In [64]:
# merging df into a new df
df_reddito = reddito_regione.merge(reddito_età, how='inner', left_on='Territorio', right_on='Territorio', suffixes=('_occupazione', '_età'))
df_reddito = df_reddito.merge(reddito_sesso, how='inner', left_on='Territorio', right_on='Territorio', suffixes=('_X', '_Y'))
df_reddito = df_reddito.rename(columns={'Value':'Value_sesso'})

# merging df
df_reddito = df_reddito.merge(reddito_titolo_studio, how='inner', left_on='Territorio', right_on='Territorio', suffixes=('_X', '_Y'))
df_reddito = df_reddito.rename(columns={'Value':'Value_titolo_studio'})

df_reddito.head(3)

,Territorio,Fonte principale di reddito familiare,Value_occupazione,Classe di età del principale percettore,Value_età,Sesso del principale percettore,Value_sesso,Titolo di studio del principale percettore,Value_titolo_studio
0,Nord-ovest,lavoro dipendente,41438.0,fino a 35 anni,33796,maschi,42031,"licenza di scuola elementare, nessun titolo di...",21444
1,Nord-ovest,lavoro dipendente,41438.0,fino a 35 anni,33796,maschi,42031,licenza di scuola media,30589
2,Nord-ovest,lavoro dipendente,41438.0,fino a 35 anni,33796,maschi,42031,diploma,38923


### DATA CLEANING

In [65]:
# adding new column
df_reddito['Reddito Medio'] = (df_reddito['Value_occupazione']+df_reddito['Value_età']+df_reddito['Value_sesso']+df_reddito['Value_titolo_studio'])/4

df_reddito['Reddito Medio'].describe()

count     1000.000000
mean     32409.830000
std       6180.542469
min      18061.000000
25%      27342.812500
50%      32424.000000
75%      36810.187500
max      49985.750000
Name: Reddito Medio, dtype: float64

In [66]:
# renaming and reordering columns

df_reddito = df_reddito.rename(columns={'Fonte principale di reddito familiare ':'tipo_reddito',
                                        'Classe di età del principale percettore':'Età_Fasce_reddito',
                                        'Titolo di studio del principale percettore':'Istruzione',
                                        'Sesso del principale percettore ':'Sesso',
                                        'Territorio':'Area_Nielsen_OLD'                                       
                                        })

df_reddito = df_reddito[['Area_Nielsen_OLD', 'tipo_reddito', 'Età_Fasce_reddito','Sesso', 'Istruzione','Reddito Medio']]

In [67]:
df_reddito.head()

,Area_Nielsen_OLD,tipo_reddito,Età_Fasce_reddito,Sesso,Istruzione,Reddito Medio
0,Nord-ovest,lavoro dipendente,fino a 35 anni,maschi,"licenza di scuola elementare, nessun titolo di...",34677.25
1,Nord-ovest,lavoro dipendente,fino a 35 anni,maschi,licenza di scuola media,36963.50
2,Nord-ovest,lavoro dipendente,fino a 35 anni,maschi,diploma,39047.00
3,Nord-ovest,lavoro dipendente,fino a 35 anni,maschi,laurea e post-laurea,43350.50
4,Nord-ovest,lavoro dipendente,fino a 35 anni,femmine,"licenza di scuola elementare, nessun titolo di...",31902.75


In [68]:
# changing values naming according to the names in the df_indagine

map_età = {'fino a 35 anni': '18-34', '35-44 anni': '35-44', '45-54 anni': '45-54', '55-64 anni': '55-64', '65 anni e più': '65+'}
df_reddito['Età_Fasce_reddito'] = df_reddito['Età_Fasce_reddito'].replace(map_età)


map_sesso = {'maschi': 'Un uomo', 'femmine': 'Una donna'}
df_reddito['Sesso'] = df_reddito['Sesso'].replace(map_sesso)


map_istruzione = {'licenza di scuola elementare, nessun titolo di studio': 'Licenza di scuola elementare',
                 'licenza di scuola media': 'Diploma di scuola media inferiore', 
                 'diploma': 'Diploma di scuola media superiore', 
                 'laurea e post-laurea': 'Laurea o dottorato', 
                 }
df_reddito['Istruzione'] = df_reddito['Istruzione'].replace(map_istruzione)


map_tipo_reddito = {'lavoro dipendente': 'dipendente',
                 'lavoro autonomo': 'autonomo', 
                 'pensioni e trasferimenti pubblici': 'pensione',
                 'altro ':'altro', 
                 }
df_reddito['tipo_reddito'] = df_reddito['tipo_reddito'].replace(map_tipo_reddito)


map_territorio = {'Nord-ovest': 'Nordovest',
                 'Nord-est': 'Nordest', 
                 }
df_reddito['Area_Nielsen_OLD'] = df_reddito['Area_Nielsen_OLD'].replace(map_territorio)

In [69]:
# dropping rows
df_reddito = df_reddito[df_reddito['tipo_reddito']!='totale']

In [70]:
print(df_reddito.shape)
df_reddito.head()

(800, 6)


,Area_Nielsen_OLD,tipo_reddito,Età_Fasce_reddito,Sesso,Istruzione,Reddito Medio
0,Nordovest,dipendente,18-34,Un uomo,Licenza di scuola elementare,34677.25
1,Nordovest,dipendente,18-34,Un uomo,Diploma di scuola media inferiore,36963.50
2,Nordovest,dipendente,18-34,Un uomo,Diploma di scuola media superiore,39047.00
3,Nordovest,dipendente,18-34,Un uomo,Laurea o dottorato,43350.50
4,Nordovest,dipendente,18-34,Una donna,Licenza di scuola elementare,31902.75


In [71]:
df_reddito['Reddito Medio'].describe()

count      800.000000
mean     32337.072500
std       6295.177456
min      18061.000000
25%      27348.812500
50%      32311.625000
75%      36803.562500
max      49985.750000
Name: Reddito Medio, dtype: float64

## FINAL DF

In [72]:
df_final = df.copy()

### MERGE

DF + DF REDDITO

adding the column "tipo_reddito" from df_reddito to df_final

In [73]:
# adding new column with values according to 'stato_occupazione_etichetta' and 'ruolo_lavoro_etichetta' columns
df_final['tipo_reddito'] = df_final['ruolo_lavoro_etichetta'].apply(lambda x: 'autonomo' if x == 'Libero professionista' else np.nan)
df_final.loc[df_final['ruolo_lavoro_etichetta'].isin(['Libero professionista','Imprenditore']), 'tipo_reddito'] = 'autonomo'
df_final.loc[df_final['stato_occupazione_etichetta'].isin(['Sei in pensione']), 'tipo_reddito'] = 'pensione'
df_final.loc[df_final['stato_occupazione_etichetta'].isin(['Stai studiando','Sei in cerca di lavoro', 'Altro','Ti occupi della famiglia/ casa a tempo pieno', np.nan]), 'tipo_reddito'] = 'nessun reddito'
df_final['tipo_reddito'].fillna('dipendente', inplace=True)

In [74]:
# checking the results
df_final[['stato_occupazione_etichetta', 'ruolo_lavoro_etichetta', 'tipo_reddito']].drop_duplicates()

,stato_occupazione_etichetta,ruolo_lavoro_etichetta,tipo_reddito
0,Lavori a tempo pieno,Quadro,dipendente
1,Lavori a tempo pieno,Altro ruolo e cioè:,dipendente
2,Lavori a tempo pieno,Impiegato,dipendente
3,NaN,NaN,nessun reddito
4,Sei in pensione,NaN,pensione
5,Lavori part-time/ saltuariamente,Impiegato,dipendente
8,Lavori part-time/ saltuariamente,Altro ruolo e cioè:,dipendente
10,Lavori part-time/ saltuariamente,Dirigente/ manager,dipendente
11,Stai studiando,NaN,nessun reddito
12,Sei in cerca di lavoro,NaN,nessun reddito


In [75]:
# adding new column for age bins according to df_reddito
df_final['Età_Fasce_reddito'] = df_final['Età_Fasce']
df_final['Età_Fasce_reddito'].replace('18-24', '18-34', inplace=True)
df_final['Età_Fasce_reddito'].replace('25-34', '18-34', inplace=True)

In [76]:
# checking df_final's columns
df_final[['Sesso', 'Età_Fasce_reddito', 'Istruzione', 'Area_Nielsen_OLD','tipo_reddito']].head()

,Sesso,Età_Fasce_reddito,Istruzione,Area_Nielsen_OLD,tipo_reddito
0,Una donna,35-44,Laurea o dottorato,Isole,dipendente
1,Un uomo,55-64,Diploma di scuola media inferiore,Nordovest,dipendente
2,Una donna,45-54,Laurea o dottorato,Nordest,dipendente
3,Un uomo,65+,Diploma di scuola media superiore,Sud,nessun reddito
4,Un uomo,65+,Diploma di scuola media inferiore,Sud,pensione


In [77]:
# checking df_reddito's columns
df_reddito.head()

,Area_Nielsen_OLD,tipo_reddito,Età_Fasce_reddito,Sesso,Istruzione,Reddito Medio
0,Nordovest,dipendente,18-34,Un uomo,Licenza di scuola elementare,34677.25
1,Nordovest,dipendente,18-34,Un uomo,Diploma di scuola media inferiore,36963.50
2,Nordovest,dipendente,18-34,Un uomo,Diploma di scuola media superiore,39047.00
3,Nordovest,dipendente,18-34,Un uomo,Laurea o dottorato,43350.50
4,Nordovest,dipendente,18-34,Una donna,Licenza di scuola elementare,31902.75


In [78]:
# adding df_reddito's columns to df_final
df_final = pd.merge(df_final, df_reddito, on=['Sesso', 'Età_Fasce_reddito', 'Istruzione', 'Area_Nielsen_OLD','tipo_reddito'], how='left')

In [79]:
df_final['Reddito Medio'].describe()

count     5789.000000
mean     36316.309250
std       5559.883224
min      22765.500000
25%      31957.750000
50%      36766.250000
75%      41059.250000
max      49985.750000
Name: Reddito Medio, dtype: float64

In [80]:
# check the results (showing unique rows)
df_final[['Sesso', 'Età_Fasce_reddito', 'Istruzione', 'Area_Nielsen_OLD','tipo_reddito','Reddito Medio']].drop_duplicates()

,Sesso,Età_Fasce_reddito,Istruzione,Area_Nielsen_OLD,tipo_reddito,Reddito Medio
0,Una donna,35-44,Laurea o dottorato,Isole,dipendente,30707.25
1,Un uomo,55-64,Diploma di scuola media inferiore,Nordovest,dipendente,39403.50
2,Una donna,45-54,Laurea o dottorato,Nordest,dipendente,42855.25
3,Un uomo,65+,Diploma di scuola media superiore,Sud,nessun reddito,NaN
4,Un uomo,65+,Diploma di scuola media inferiore,Sud,pensione,25495.25
...,...,...,...,...,...,...
8055,Una donna,18-34,Diploma di scuola media inferiore,Isole,dipendente,23703.25
8059,Un uomo,18-34,Diploma di scuola media inferiore,Nordovest,nessun reddito,NaN
8070,Una donna,55-64,Diploma di scuola media inferiore,Nordovest,autonomo,39516.50
8073,Una donna,35-44,Diploma di scuola media inferiore,Isole,pensione,23996.50


### ADDING MORE VARIABLES

Adding the following columns:

Abitanti_Classi (valore), p_Q9 (istruzione valore), B03_Q62 (dimensione nucleo familiare), B03_Q63 (n. bambini in famiglia), B03_Q66 (spesa media mensile)

In [81]:
# adding the column 'istruzione_valore'
df_final = df_final.merge(indagine[['ID_Rispondente','p_Q9']], how='left', left_on='ID_Rispondente', right_on='ID_Rispondente')
df_final = df_final.rename(columns={'p_Q9':'istruzione_valore'})

# adding the column 'Abitanti_classi_valore'
df_final = df_final.merge(indagine[['ID_Rispondente','Abitanti_Classi']], how='left', left_on='ID_Rispondente', right_on='ID_Rispondente')
df_final = df_final.rename(columns={'Abitanti_Classi_x':'Abitanti_Classi'})
df_final = df_final.rename(columns={'Abitanti_Classi_y':'Abitanti_Classi_valore'})

In [82]:
# adding the column 'nucleo_familiare'
df_final = df_final.merge(profilo[['ID_Rispondente','B03_Q62']], how='left', left_on='ID_Rispondente', right_on='ID_Rispondente')
df_final = df_final.rename(columns={'B03_Q62':'nucleo_familiare'})

# adding the column 'numero_bambini_famiglia'
df_final = df_final.merge(profilo[['ID_Rispondente','B03_Q63']], how='left', left_on='ID_Rispondente', right_on='ID_Rispondente')
df_final = df_final.rename(columns={'B03_Q63':'numero_bambini_famiglia'})

# adding the column 'spesa_media_mensile_famiglia'
df_final = df_final.merge(profilo[['ID_Rispondente','B03_Q66']], how='left', left_on='ID_Rispondente', right_on='ID_Rispondente')
df_final = df_final.rename(columns={'B03_Q66':'spesa_media_mensile_famiglia'})

In [83]:
# adding the column 'nucleo_familiare_etichetta'
df_final = df_final.merge(profilo_labels[['ID_Rispondente','B03_Q62']], how='left', left_on='ID_Rispondente', right_on='ID_Rispondente')
df_final = df_final.rename(columns={'B03_Q62':'nucleo_familiare_etichetta'})

# adding the column 'numero_bambini_famiglia_etichetta'
df_final = df_final.merge(profilo_labels[['ID_Rispondente','B03_Q63']], how='left', left_on='ID_Rispondente', right_on='ID_Rispondente')
df_final = df_final.rename(columns={'B03_Q63':'numero_bambini_famiglia_etichetta'})

# adding the column 'spesa_media_mensile_famiglia_etichetta'
df_final = df_final.merge(profilo_labels[['ID_Rispondente','B03_Q66']], how='left', left_on='ID_Rispondente', right_on='ID_Rispondente')
df_final = df_final.rename(columns={'B03_Q66':'spesa_media_mensile_famiglia_etichetta'})

## DOWNLOAD CSV

In [84]:
print(df_final.shape)
df_final.head()

(8128, 39)


,ID_Rispondente,futuro,oggi,futuro_pesato,oggi_pesato,Tempo compilazione,Secondi per risposta,Data compilazione,Sesso,Age,...,Età_Fasce_reddito,Reddito Medio,istruzione_valore,Abitanti_Classi_valore,nucleo_familiare,numero_bambini_famiglia,spesa_media_mensile_famiglia,nucleo_familiare_etichetta,numero_bambini_famiglia_etichetta,spesa_media_mensile_famiglia_etichetta
0,256848,7,11,1.528314,2.401636,98.013,8.167750,2023-02-19,Una donna,39.0,...,35-44,30707.25,1.0,3.0,3.0,2.0,3.0,3 persone,"Sì, è presente 1 bambino 0-14 anni",Tra 2.000 e 3.000 Euro
1,337988,1,9,1.211879,10.906914,145.659,12.138250,2023-05-12,Un uomo,56.0,...,55-64,39403.50,3.0,2.0,6.0,2.0,2.0,6 persone o più,"Sì, è presente 1 bambino 0-14 anni",Tra 1.000 e 2.000 Euro
2,371674,1,2,0.218331,0.436661,143.891,11.990917,2023-02-28,Una donna,46.0,...,45-54,42855.25,1.0,4.0,3.0,2.0,2.0,3 persone,"Sì, è presente 1 bambino 0-14 anni",Tra 1.000 e 2.000 Euro
3,379221,13,10,38.273890,29.441454,237.097,19.758083,2023-04-22,Un uomo,77.0,...,65+,NaN,2.0,2.0,2.0,1.0,2.0,2 persone,"No, nel mio nucleo non ci sono bambini 0-14 anni",Tra 1.000 e 2.000 Euro
4,514191,10,9,20.472955,18.425660,258.143,21.511917,2023-03-10,Un uomo,73.0,...,65+,25495.25,3.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN


In [85]:
df_final.to_csv('data_tableau/indagine.csv', index=False)